In [1]:
import pandas as pd
import pickle


In [2]:
%pwd

'/Users/ronjaboehmer/code/mariajoi/flightdelay/notebooks'

In [3]:
%cd ../flightdelay/utils

/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/utils


/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from mytrans import MyTrans
trans = MyTrans()

# Test with refit

In [19]:
df = pd.read_csv("../data/130k_zeros_removed.csv")
df = df.drop("Unnamed: 0", axis = 1)

In [20]:

X = df.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups',
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

X.head(5)


,Month,DayOfWeek,Operating_Airline,Origin,Dest,DepTimeBlk,ArrTimeBlk,DistanceGroup
0,1,7,AA,CMH,MIA,0600-0659,0900-0959,4
1,1,5,WN,AUS,ELP,1900-1959,2000-2059,3
2,1,7,YX,GSO,LGA,0600-0659,0800-0859,2
3,1,3,B6,LAS,JFK,1300-1359,2100-2159,9
4,1,2,AA,ORD,LGA,1300-1359,1600-1659,3


In [21]:
trans.fit(X)
X_trans =trans.transform(X)
X_trans.columns

Index(['Operating_Airline', 'Origin', 'Dest', 'DistanceGroup', 'Month_sin',
       'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'groups_DepTimeBlk',
       'groups_ArrTimeBlk'],
      dtype='object')

In [8]:
X_trans.dtypes

Operating_Airline     object
Origin                object
Dest                  object
DistanceGroup          int64
Month_sin            float64
Month_cos            float64
DayOfWeek_sin        float64
DayOfWeek_cos        float64
groups_DepTimeBlk     object
groups_ArrTimeBlk     object
dtype: object

# Pipelines

In [5]:
%cd ../pickle

/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/pickle


/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Pipeline 1 test

In [6]:
pipeline = pickle.load(open("fitted_model_pipeline_test.pkl","rb"))

pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x129000940>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=5, random_state=42))])

In [25]:
df["BadFlight"] = ((df['ArrivalDelayGroups'] > 0) | (df['Cancelled'] > 0)).astype(int)

y_bin = df["BadFlight"]

In [26]:
pipeline.fit(X_trans, y_bin)

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x10e2e0fa0>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=5, random_state=42))])

## Pipeline 2 test

In [7]:
pipeline2 = pickle.load(open("fitted_model_pipeline_vs2.pkl","rb"))

pipeline2

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1257fe1a0>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=5, random_state=42))])

# Test on new X value

In [8]:
X_pred = pd.DataFrame({"Operating_Airline": "9E",
                           "Origin": "SHV",
                           "Dest": "ATL",
                           "DepTimeBlk": "1500-1559",
                           "ArrTimeBlk": "1800-1859",
                           "DayOfWeek": 4,
                           "Month": 1,
                           "DistanceGroup": 3}, index = [0])


In [9]:
X_pred

,Operating_Airline,Origin,Dest,DepTimeBlk,ArrTimeBlk,DayOfWeek,Month,DistanceGroup
0,9E,SHV,ATL,1500-1559,1800-1859,4,1,3


In [10]:
trans.fit(X_pred)
X_trans_pred = trans.transform(X_pred)


In [11]:
pipeline.predict(X_trans_pred)


ValueError: X has 10 features, but ColumnTransformer is expecting 775 features as input.

In [11]:
pipeline2.predict(X_trans_pred)


array([1])